# Cell

Problem:

1. In GDS each cell must have a unique name.
2. In GDS two cells stored in the GDS file cannot have the same name.

Solution: The decorator `@gf.cell` fixes both issues:

1. By giving the cell a unique name depending on the parameters that you pass
2. By creating a cache of cells where we use the cell name as the key. This to create two cells with the same name in the same python script.

Lets see how it works

In [ ]:
import gdsfactory as gf

@gf.cell
def wg(length=10, width=1):
    c = gf.Component()
    c.add_polygon([(0, 0), (length, 0), (length, width), (0, width)], layer=(1, 0))
    c.add_port(name='o1', midpoint=[0, width / 2], width=width, orientation=180)
    c.add_port(name='o2', midpoint=[length, width / 2], width=width, orientation=0)
    return c

See how the cells get the name from the parameters that you pass them

In [ ]:
c = wg()
print(c)

c = wg(width=0.5)
print(c)

How can you have add two different references to a cell with the same parameters?

In [ ]:
# Problem
import gdsfactory as gf

c = gf.Component('problem')
R1 = gf.components.rectangle(size=(4,2))  # Creates a rectangle (same Unique ID uid)
R2 = gf.components.rectangle(size=(4,2))
# Try Create a new rectangle that we want to change (but has the same name so we will get R1 from the cache)

r1r = c << R1  # Add the first rectangle to c
r2r = c << R2  # Add the second rectangle to c
c

In [ ]:
print(R1 == R2)
print(R1)
print(R2)

# But now I want to rotate R2 -- I can't because it doesn't exist! The error is that it rotates both references!
R2.rotate(45)
# I think I'm rotating a second rectangle, but actually R2 points to R1 even though I specifically tried to create two rectangles
c.plot()

if you run the cell above you will see the cell rotating. This is a bad way to manipulate cells. That's why wrote the references tutorial.

In [ ]:
# Solution: use references
import gdsfactory as gf

c = gf.Component('solution')
R = gf.components.rectangle(size=(4,2))  

r1 = c << R  # Add the first rectangle reference to c
r2 = c << R  # Add the second rectangle reference to c

r2.rotate(45)
c

## Adding port markers

When you use `component.show(show_ports=True)` it automatically adds pins markers to all ports

In [ ]:
import gdsfactory as gf
c = gf.components.straight()
c.show(show_ports=True)
c.plot()

We can even show ports of all references with `component.show(show_subports=True)`

In [ ]:
c = gf.components.mzi_phase_shifter(length_x=50)
c

## Cache

To avoid that 2 exact cells are not references of the same cell the `cell` decorator has a
cache where if component has already been build it will return the component
from the cache


In [ ]:
@gf.cell
def wg(length=10, width=1):
    c = gf.Component()
    c.add_polygon([(0, 0), (length, 0), (length, width), (0, width)], layer=(1, 0))
    print("calling wg function")
    return c

In [ ]:
wg1 = wg()  # cell builds a straight
print(wg1)

In [ ]:
wg2 = wg()  # cell returns the same straight as before without having to run the function
print(wg2) # notice that they have the same uuid (unique identifier)

In [ ]:
wg2.plot()

In [ ]:
from gdsfactory.cell import print_cache

Lets say that you change the code of the straight function in a jupyter notebook like this one.  (I mostly use Vim/VsCode/Pycharm for creating new cells in python)

In [ ]:
print_cache() # Cache gets clear with each show or plot just to be safe

In [ ]:
wg3 = wg()
wg4 = wg(length=11)

In [ ]:
print_cache() 

In [ ]:
wg3.plot()

To enable nice notebook tutorials, every time we show a cell in Matplotlib or Klayout, we clear the cache, in case you want to develop cells in jupyter notebooks or an IPython kernel

In [ ]:
print_cache() # cache is now empty

## Validate argument types

To make sure you pass the correct arguments to the cell it runs a validator that checks the type annotations for the function.


For example this will be correct

```python
import gdsfactory as gf

@gf.cell
def straigth_waveguide(length:float):
    return gf.components.straigth(length=length)


component = straigth_waveguide(length=3)
```


While this will raise an error

```python
import gdsfactory as gf

@gf.cell
def straigth_waveguide(length:float):
    return gf.components.straigth(length=length)


component = straigth_waveguide(length='long')
```

by default `@cell` validates all arguments using [pydantic](https://pydantic-docs.helpmanual.io/usage/validation_decorator/#argument-types)